## Simple 2D Simulation

Outline:
- Import helpers
- Build a 2D simulation and initial state
- Plot initial configuration with labels
- Step the simulation and plot $N_m(t)$
- Replot the configuration after stepping

In [ ]:
# Imports for the simple simulation section
import importlib
import numpy as np
import plotly.graph_objects as go
import hard_spheres

importlib.reload(hard_spheres)
from hard_spheres import HardSpheresSimulation

In [ ]:
def build_circle_trace(
  coords: np.ndarray,
  r: float,
  color: str = "black",
  width: int = 1,
  n_points: int = 48,
) -> go.Scatter:
  theta = np.linspace(0.0, 2.0 * np.pi, n_points)
  xs: list[float | None] = []
  ys: list[float | None] = []
  for cx, cy in coords:
    xs.extend(cx + r * np.cos(theta))
    ys.extend(cy + r * np.sin(theta))
    xs.append(None)
    ys.append(None)
  return go.Scatter(
    x=xs,
    y=ys,
    mode="lines",
    line=dict(color=color, width=width),
    showlegend=False,
    hoverinfo="skip",
  )


def plot_box_with_spheres_2d(
  coords: np.ndarray,
  Lbox: float,
  r: float,
  title: str,
  annotate: bool = True,
) -> go.Figure:
  half = Lbox * 0.5
  fig = go.Figure()
  fig.add_shape(
    type="rect",
    x0=-half,
    x1=half,
    y0=-half,
    y1=half,
    line=dict(color="gray", width=1),
  )

  pastel_palette = [
    "rgba(255, 179, 186, 0.45)",
    "rgba(186, 225, 255, 0.45)",
    "rgba(186, 255, 201, 0.45)",
    "rgba(255, 223, 186, 0.45)",
    "rgba(204, 204, 255, 0.45)",
  ]
  theta = np.linspace(0.0, 2.0 * np.pi, 48)
  for idx, (cx, cy) in enumerate(coords):
    if idx % 10 != 0:
      continue
    color = pastel_palette[(idx // 10) % len(pastel_palette)]
    fig.add_trace(
      go.Scatter(
        x=cx + r * np.cos(theta),
        y=cy + r * np.sin(theta),
        mode="lines",
        fill="toself",
        fillcolor=color,
        line=dict(color=color, width=1),
        showlegend=False,
        hoverinfo="skip",
      )
    )

  fig.add_trace(build_circle_trace(coords, r))

  if annotate:
    fig.add_trace(
      go.Scatter(
        x=coords[:, 0],
        y=coords[:, 1],
        mode="text",
        text=[str(i) for i in range(coords.shape[0])],
        textposition="middle center",
        textfont=dict(size=8, color="black"),
        showlegend=False,
        hoverinfo="skip",
      )
    )

  fig.update_layout(
    title=title,
    width=1600,
    height=1600,
    plot_bgcolor="white",
  )
  fig.update_xaxes(range=[-half, half], visible=False)
  fig.update_yaxes(range=[-half, half], scaleanchor="x", scaleratio=1, visible=False)
  return fig


def plot_nm_over_time(
  times: list[int],
  nm_values: list[float],
  title: str,
  y_label: str = "Nm",
) -> go.Figure:
  fig = go.Figure()
  fig.add_trace(
    go.Scatter(
      x=times,
      y=nm_values,
      mode="lines+markers",
      line=dict(color="black", width=2),
      marker=dict(size=6),
      showlegend=False,
    )
  )
  fig.update_layout(
    title=title,
    width=800,
    height=450,
    plot_bgcolor="white",
    xaxis_title="t",
    yaxis_title=y_label,
  )
  return fig

In [ ]:
# Build a 2D simulation with random initialization
sim_params = {
  "dim": 2,
  "Lbox": 20.0,
  "n_spheres": 260,
  "r": 0.5,
  "dl": 0.2,
  "seed": 42,
}
sim = HardSpheresSimulation(
  dim=sim_params["dim"],
  Lbox=sim_params["Lbox"],
  r=sim_params["r"],
  dl=sim_params["dl"],
  history_enabled=True,
  cache_history=True,
  cache_stride=1000,
  use_numba=True,
 )
sim.SetInitialConditions(
  coords=None,
  N=sim_params["n_spheres"],
  randomInit=True,
  max_rand_init_iterations=100000,
  seed=sim_params["seed"],
)

plot_box_with_spheres_2d(
  coords=sim.coords,
  Lbox=sim_params["Lbox"],
  r=sim_params["r"],
  title="Initial configuration (2D)",
).show(renderer="png")

In [ ]:
# Step the simulation and track Nm over time
n_steps = 5000
for step in range(n_steps):
  sim.TimeStep(seed=sim_params["seed"] + step)
histories = sim.history
times = [h.time for h in histories]
nm_percent = [100.0 * h.Nm / sim.N for h in histories]

mean_free_path = sim.MeanPseudoFreePath(
  n_directions=30,
  step=sim_params["dl"],
  seed=sim_params["seed"] + 1000,
 )
print(f"Mean pseudo free path after {n_steps} steps: {mean_free_path:.4f}")

plot_nm_over_time(
  times,
  nm_percent,
  title="Moves accepted per time step",
  y_label="Nm/N (%)",
).show(renderer="png")

In [ ]:
plot_box_with_spheres_2d(
  coords=sim.coords,
  Lbox=sim_params["Lbox"],
  r=sim_params["r"],
  title="Configuration after 5000 steps (2D)",
).show(renderer="png")